<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/Py_trodes/PyTrodes_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
New Python Based Pipeline for Trodes
"""

In [ ]:
""" Generate RAW_LFP Binaries from *.rec files"""
""" Import RAW_LFP """
""" Import XY_position Data """
""" Find Ripples """
""" Map Channels """
""" Finde APs """
""" Find IIS """


In [8]:
import os
import subprocess
import numpy as np

In [ ]:
""" Generate RAW_LFP Binaries from *.rec files"""
dest_dir = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS\RAW_DATA'
export_func_location = r'C:\Users\dennis.tabuena\Desktop\_SPIKE_GADGETS\Trodes_2-5-1_Windows64\Trodes_2-5-1_Windows64\trodesexport.exe'
dot_rec_file_loc = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\Test_Emily\AngelfoodCake01\AngelfoodCake01.rec'
full_cmd_raw = ' '.join([export_func_location, '-raw',
                    '-rec ', dot_rec_file_loc,
                    '-userawrefs', str(0),
                    '-outputdirectory', dest_dir])
export_report = subprocess.run(full_cmd_raw)



' Find IIS '

In [50]:
""" Import RAW_LFP """
current = os.getcwd()
trodes_dat_reader_loc = r'C:\Users\dennis.tabuena\Desktop\_SPIKE_GADGETS\Trodes_2-5-1_Windows64\Trodes_2-5-1_Windows64\Resources\TrodesToPython'
os.chdir(trodes_dat_reader_loc)
import readTrodesExtractedDataFile3 as trodes
os.chdir(current)

dot_dat_file_loc = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS\RAW_DATA\AngelfoodCake01.raw\AngelfoodCake01.raw_group0.dat'
raw_lfp = trodes.readTrodesExtractedDataFile(dot_dat_file_loc)




In [60]:
raw_lfp['data'][0]

array([ -941, -2416, -2515,  -896,  -800, -1928, -2480, -1313,  -766,
       -1211,  -555,  -688, -2470, -2975, -2786, -1812,  -282,   -44,
         820, -2778, -1690, -2872, -1263, -2487,  -719,   434,   888,
       -3760, -2526, -1521, -2508, -2445], dtype=int16)

In [58]:
# raw_lfp['data'] = [t[0] for t in raw_lfp['data']]
# raw_lfp['data'] = np.stack(raw_lfp['data'],dtype='float16')

MemoryError: 